In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import scipy.io
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

dataset = scipy.io.loadmat('/content/drive/My Drive/spatialpyramidfeatures4scene15.mat')
feature_mat = dataset['featureMat']
label_mat = dataset['labelMat']
label_mat = np.argwhere(label_mat == 1)[:, 0:1].T

##################################################
##### First Layer                            #####
##################################################

# Step 1
labels = np.unique(label_mat)
train_per_image = 100

train_feature = None
test_feature = None

train_label = None
test_label = None

for label in labels:
    indices_per_label = np.argwhere(label_mat == label)[:, 1]
    np.random.shuffle(indices_per_label)
    train_index, test_index = indices_per_label[:train_per_image], indices_per_label[train_per_image:]
    
    if train_feature is None:
        train_feature = feature_mat[:, train_index]
    else:
        train_feature = np.concatenate([train_feature, feature_mat[:, train_index]], axis=1)
    
    if test_feature is None:
        test_feature = feature_mat[:, train_index]
    else:
        test_feature = np.concatenate([test_feature, feature_mat[:, train_index]], axis=1)    

    if train_label is None:
        train_label = label_mat[:, train_index]
    else:
        train_label = np.concatenate([train_label, label_mat[:, train_index]], axis=1)
    
    if test_label is None:
        test_label = label_mat[:, train_index]
    else:
        test_label = np.concatenate([test_label, label_mat[:, train_index]], axis=1)  

data_channels = 1
subnetwork_nodes = 5
d = 100
ld = 1
abs_error = 0.2
n, M = train_feature.shape
H = None

ah_array = []
bh_array = []

for data_channel in range(data_channels):
    ah_array.append([])
    bh_array.append([])
    
    for i in range(subnetwork_nodes):
        # Step 2
        x = train_feature
        af = np.random.uniform(size=(n, d)) * 2 - 1
        bf = np.random.uniform(size=(d, 1))
        Hf = np.dot(af.T, x) + bf
        
        # Step 3
        y = train_label
        ah = np.dot(y, np.linalg.pinv(Hf)).T
        
        mse = mean_squared_error(np.dot(ah.T, Hf), y)
        bh = np.sqrt(mse)
        
        # Step 4
        e = y - (np.dot(ah.T, Hf) + bh)
        
        # Step 5
        P = np.dot(np.linalg.pinv(ah.T), e)
        
        # Step 6
        P = P + Hf
        
        # Step 7
        atemp = np.dot(P, np.linalg.pinv(x)).T
        af = af + ld * (atemp - af)
        
        mse = mean_squared_error(np.dot(af.T, x), P)
        bf = np.sqrt(mse)
        
        # Step 8
        Hf = np.dot(af.T, x) + bf
        
        ##################################################
        ##### Second Layer                           #####
        ##################################################
        
        if H is None:
            H = Hf
        else:
            H = np.concatenate((H, Hf))
        
        ah_array[data_channel].append(af)
        bh_array[data_channel].append(bf)

##################################################
##### Third Layer                            #####
##################################################

subnetwork_nodes = 5
e = train_label

def h(x):
    return np.sin(x)
    #return 1 / (1 + np.exp(x))

def hinv(x):
    return np.arcsin(x)
    #return -np.log(1 / x - 1)

def u(x):
    tempX = x / 20
    tempX[tempX > 1] = 1
    tempX[tempX < -1] = -1
    return tempX

def uinv(x):
    return x * 20

ap_array = []
bp_array = []
betap_array = []

for i in range(subnetwork_nodes):
    #scaler = MinMaxScaler(feature_range=(0.1,0.9))
    #scaler.fit(e.T)
    #ue = scaler.transform(e)
    
    ap = np.dot(hinv(u(e)), np.linalg.pinv(H)).T
    bp = np.average(np.dot(ap.T, H) - hinv(u(e)))
    #mse = mean_squared_error(np.dot(ap.T, H), hinv(u(e)))
    #bp = np.sqrt(mse)
    
    tempE = uinv(h(np.dot(ap.T, H) + bp))
    betap = np.array([np.einsum('ij,ij->i', e, tempE) / np.einsum('ij,ij->i', tempE, tempE)]).T
    
    e = e - betap * tempE
    
    ap_array.append(ap)
    bp_array.append(bp)
    betap_array.append(betap)    

##################################################
##### Test                                   #####
##################################################

n, M = test_feature.shape
H = None

for i in range(data_channels):
    for j in range(subnetwork_nodes):
        x = test_feature
        Hf = np.dot(ah_array[i][j].T, x) + bh_array[i][j]
        
        if H is None:
            H = Hf
        else:
            H = np.concatenate((H, Hf))       
        
t = None
for i in range(subnetwork_nodes):
    if t is None:
        t = betap_array[i] * uinv(h(np.dot(ap_array[i].T, H) + bp_array[i]))
    else:
        t = t + betap_array[i] * uinv(h(np.dot(ap_array[i].T, H) + bp_array[i]))

correct = np.count_nonzero(test_label - t < abs_error)
total = test_label.shape[1]
accuracy = correct / total * 100

print("test accuracy: " + "{0:0.2f}".format(accuracy))

test accuracy: 100.00
